In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.special import expit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude,longitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,current.short_rad,current.diff_rad,current.dni,current.gti,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1757822614,2025-09-14 01:03,1757822400,2025-09-14 01:00,10.4,50.7,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003,6.3,10.1,87,E,1024.0,30.24,0.0,0.0,94,75,9.1,48.4,8.2,46.7,9.6,49.3,7.2,45.0,10.0,6.0,0.0,8.7,13.9,0,0,0,0,"[{'date': '2025-09-14', 'date_epoch': 17578080..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-14,1757808000,"[{'time_epoch': 1757818800, 'time': '2025-09-1...",25.0,77.0,8.4,47.1,15.0,59.0,7.6,12.2,0.0,0.0,0.0,10.0,6.0,67,0,0,0,0,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,2.1,06:15 AM,06:10 PM,12:48 AM,11:14 AM,Last Quarter,55,0,0


In [5]:
hour_list = dados["forecast"]["forecastday"][0]["hour"]
df_hour = pd.json_normalize(hour_list)
df_hour


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,short_rad,diff_rad,dni,gti,condition.text,condition.icon,condition.code
0,1757818800,2025-09-14 00:00,9.9,49.9,0,6.5,10.4,87,E,1023.0,30.21,0.0,0.0,0.0,84,34,8.5,47.3,8.5,47.3,9.9,49.9,7.4,45.4,0,0,0,0,10.0,6.0,8.7,14.0,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
1,1757822400,2025-09-14 01:00,10.4,50.7,0,6.3,10.1,87,E,1024.0,30.24,0.0,0.0,0.0,94,75,8.2,46.7,8.2,46.7,9.6,49.3,7.2,45.0,0,0,0,0,10.0,6.0,8.7,13.9,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
2,1757826000,2025-09-14 02:00,9.2,48.6,0,5.6,9.0,85,E,1022.0,30.18,0.0,0.0,0.0,87,35,7.9,46.1,7.9,46.1,9.2,48.6,7.1,44.8,0,0,0,0,10.0,6.0,7.9,12.7,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
3,1757829600,2025-09-14 03:00,8.8,47.9,0,5.4,8.6,86,E,1022.0,30.17,0.0,0.0,0.0,88,37,7.5,45.5,7.5,45.5,8.8,47.9,6.9,44.5,0,0,0,0,10.0,6.0,7.9,12.7,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
4,1757833200,2025-09-14 04:00,8.6,47.4,0,5.1,8.3,80,E,1022.0,30.19,0.0,0.0,0.0,89,41,7.3,45.1,7.3,45.1,8.6,47.4,6.8,44.3,0,0,0,0,10.0,6.0,7.6,12.3,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
5,1757836800,2025-09-14 05:00,8.4,47.1,0,5.6,9.0,76,ENE,1023.0,30.20,0.0,0.0,0.0,89,49,6.9,44.5,6.9,44.5,8.4,47.1,6.8,44.2,0,0,0,0,10.0,6.0,8.2,13.2,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
6,1757840400,2025-09-14 06:00,9.1,48.4,0,5.4,8.6,73,ENE,1023.0,30.21,0.0,0.0,0.0,85,40,7.8,46.1,7.8,46.1,9.1,48.4,6.8,44.2,0,0,0,0,10.0,6.0,7.1,11.4,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
7,1757844000,2025-09-14 07:00,10.9,51.6,1,4.9,7.9,67,ENE,1023.0,30.22,0.0,0.0,0.0,75,4,10.1,50.2,10.1,50.2,10.9,51.6,6.6,43.9,0,0,0,0,10.0,6.0,5.7,9.1,0.2,4.06,2.10,2.68,3.34,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
8,1757847600,2025-09-14 08:00,14.1,57.3,1,4.9,7.9,64,ENE,1023.0,30.21,0.0,0.0,0.0,60,0,13.8,56.8,13.8,56.8,14.1,57.3,6.3,43.4,0,0,0,0,10.0,6.0,5.7,9.1,1.1,32.03,15.80,19.49,26.18,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
9,1757851200,2025-09-14 09:00,17.2,63.0,1,4.9,7.9,62,ENE,1022.0,30.18,0.0,0.0,0.0,46,0,17.2,63.0,17.2,63.0,17.2,63.0,5.6,42.2,0,0,0,0,10.0,6.0,5.7,9.1,3.1,106.32,26.80,90.73,79.73,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000


In [25]:
# Lista apenas com as colunas importantes
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
    "condition.text"
]

df_limpo = df_hour[colunas_importantes]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain,condition.text
15,2025-09-14 15:00,24.2,32,7,4.4,11.9,0,0,Sol
4,2025-09-14 04:00,7.3,89,41,0.0,8.3,0,0,Parcialmente nublado
3,2025-09-14 03:00,7.5,88,37,0.0,8.6,0,0,Parcialmente nublado
5,2025-09-14 05:00,6.9,89,49,0.0,9.0,0,0,Parcialmente nublado
10,2025-09-14 10:00,20.0,37,1,5.6,9.0,0,0,Sol


In [29]:
# Cria uma tabela associando estilos de roupa com dados do clima
df_tabela = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
    "condição do dia": df_limpo ["condition.text"].values[:4]
})

df_tabela

,estilos,data e hora,temperatura,chance de chuva,umidade,uv,condição do dia
0,casual,2025-09-14 00:00,8.5,0,84,0.0,Parcialmente nublado
1,esportivo,2025-09-14 01:00,8.2,0,94,0.0,Parcialmente nublado
2,social,2025-09-14 02:00,7.9,0,87,0.0,Parcialmente nublado
3,streetwear,2025-09-14 03:00,7.5,0,88,0.0,Parcialmente nublado


In [8]:
#subdados para peças de roupas por estilo especifico

subdados_roupas = {
    "casual": {
        "casual para calor": ["camisetas polo", "camisetas básicas de algodão", "camisas de manga curta", "bermudas de sarja", "bermuda jeans"],
        "casual para frio": ["suéter leve", "jaqueta jeans", "jaqueta de sarja", "calça jeans", "calça chino"],
        "casual meia estação": ["camisa xadrez", "camisa de botões leve", "cardigan fino", "calça slim"],
    },
    "esportivo": {
        "esportivo para calor": ["camisetas dry-fit", "regatas, shorts esportivos", "bermudas de tecido leve"],
        "esportivo para frio": ["moletom com capuz", "calça jogger", "calça de moletom", "jaqueta corta-vento"],
        "esportivo meia estação": ["blusa térmica fina", "calça legging", "jaqueta leve de treino"],
    },
    "social": {
        "social para calor": ["camisas de linho", "camisas de manga curta sociais", "calças de alfaiataria leve", "calça chino social"],
        "social para frio": ["blazer estruturado", "suéter de lã", "colete de alfaiataria", "calça de lã fria", "trench coat"],
        "social meia estação": ["trench coat leve", "camisa social slim fit", "calça social reta"],
    },
    "streetwear": {
        "streetwear para calor": ["camisetas oversized", "camisas estampadas leves", "bermudas cargo", "bermudas de moletom"],
        "streetwear para frio": ["hoodies com estampas", "jaquetas bomber", "jaquetas jeans oversized", "calça cargo", "moletom oversized"],
        "streetwear meia estação": ["jaqueta utilitária", "camisa xadrez oversized", "calça jogger estilizada", "jaqueta college"],
    }
}

In [15]:
print(df_tabela.columns.tolist())

['location.lat', 'location.lon', 'location.localtime_epoch', 'current.last_updated_epoch', 'current.temp_c', 'current.temp_f', 'current.is_day', 'current.condition.code', 'current.wind_mph', 'current.wind_kph', 'current.wind_degree', 'current.pressure_mb', 'current.pressure_in', 'current.precip_mm', 'current.precip_in', 'current.humidity', 'current.cloud', 'current.feelslike_c', 'current.feelslike_f', 'current.windchill_c', 'current.windchill_f', 'current.heatindex_c', 'current.heatindex_f', 'current.dewpoint_c', 'current.dewpoint_f', 'current.vis_km', 'current.vis_miles', 'current.uv', 'current.gust_mph', 'current.gust_kph', 'current.short_rad', 'current.diff_rad', 'current.dni', 'current.gti']


In [41]:
#transformando variáveis e em um número categorico 

df_tabela = pd.get_dummies(df[df_tabela])
x = pd.concat([df["current.feelslike_c"], df_tabela], axis=1)
y = df_tabela["estilos"]


KeyError: 'estilos'